# CNN without transfer
this module run CNN without using C3D. We will train the model by using the data provided by Jake without using transfer learning

In [40]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import json
%matplotlib inline

# load dataset and preprocess
 - First, run DataProprocessing Notebook to get label (Y) and data (X) txt file. 
 - Second, move label and data txt file into this notebook foler
 - third, run the code below

In [41]:
def get_data(num_training = 81, num_validation = 10, num_test = 10):
    ''' 
    load the training data provided by Jake 
    
    '''
    # load the raw data
    with open('label.json', 'r') as fp:
        y_total = json.load(fp)
        y_total = np.array(y_total)
    with open('data.json', 'r') as fp:
        x_total = json.load(fp)
        x_total = np.array(x_total)
        
    # Subsample the data
    # training data
    mask_train = range(0, num_training)
    x_train = x_total[mask_train]
    y_train = y_total[mask_train]
    # validation data
    mask_val = range(num_training, num_training + num_validation)
    x_val = x_total[mask_val]
    y_val = y_total[mask_val]
    # test data
    mask_test = range(num_training+num_validation, num_training+num_validation+num_test)
    x_test = x_total[mask_test]
    y_test = y_total[mask_test]
    
    # normalize the data: subtract the mean
    mean_video = np.floor(np.mean(x_train, axis=0))
    x_train = x_train - mean_video
    x_val = x_val - mean_video
    x_test = x_test - mean_video
    
    return x_train, y_train, x_val, y_val, x_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = get_data()

print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (81, 5, 64, 64, 3)
Train labels shape:  (81,)
Validation data shape:  (10, 5, 64, 64, 3)
Validation labels shape:  (10,)
Test data shape:  (10, 5, 64, 64, 3)
Test labels shape:  (10,)


# Some useful utilities
Remember that our image data is initially N x F x H x W x C, where:
 - N is the number of datapoints
 - F is the frame of video
 - H is the height of each frame in pixels
 - W is the height of each frame in pixels
 - C is the number of channels (usually 3: R, G, B)

# Dummy model
train a dummy model by simpily convert the data x shape to N x H x W x F, while using RGB to average all three channels
This dummy model has two layer with the first one to be a convolutional layer and the second one to be a FC layer

In [42]:
x_train = 0.21 * x_train[:,:,:,:,0] + 0.72 * x_train[:,:,:,:,1] + 0.07 * x_train[:,:,:,:,2]
x_val = 0.21 * x_val[:,:,:,:,0] + 0.72 * x_val[:,:,:,:,1] + 0.07 * x_val[:,:,:,:,2]
x_test = 0.21 * x_test[:,:,:,:,0] + 0.72 * x_test[:,:,:,:,1] + 0.07 * x_test[:,:,:,:,2]

N, F, H, W = x_train.shape
x_train = np.reshape(x_train, (N,H,W,F))
N, F, H, W = x_val.shape
x_val = np.reshape(x_val, (N,H,W,F))
N, F, H, W = x_test.shape
x_test = np.reshape(x_test, (N,H,W,F))
print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (81, 64, 64, 5)
Train labels shape:  (81,)
Validation data shape:  (10, 64, 64, 5)
Validation labels shape:  (10,)
Test data shape:  (10, 64, 64, 5)
Test labels shape:  (10,)


In [43]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 64, 64, 5])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

# define our model
def simple_model(X,y):
    # define our weights (e.g. init_two_layer_convnet)
    
    # setup variables
    Wconv1 = tf.get_variable("Wconv1", shape=[7, 7, 5, 16])
    bconv1 = tf.get_variable("bconv1", shape=[16])
    # 13456 = 32*29*29. with no padding and out dim is (64 - 7 + 0)/2 + 1 = 29
    W1 = tf.get_variable("W1", shape=[13456, 4]) 
    b1 = tf.get_variable("b1", shape=[4])

    # define our graph (e.g. two_layer_convnet)
    # valid padding means no padding
    a1 = tf.nn.conv2d(X, Wconv1, strides=[1,2,2,1], padding='VALID') + bconv1
    h1 = tf.nn.relu(a1)
    h1_flat = tf.reshape(h1,[-1,13456]) # -1 is N 
    y_out = tf.matmul(h1_flat,W1) + b1
    return y_out

y_out = simple_model(X,y)

# define our loss
total_loss = tf.losses.hinge_loss(tf.one_hot(y,4),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)

# train the model

In [44]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=40, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [48]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())
print('Training')
run_model(sess,y_out,mean_loss,x_train,y_train,20,40,10,train_step)

Training
Iteration 0: with minibatch training loss = 6.79 and accuracy of 0.2
Epoch 1, Overall loss = 13.7 and accuracy of 0.333
Epoch 2, Overall loss = 8.14 and accuracy of 0.617
Epoch 3, Overall loss = 6.46 and accuracy of 0.63
Iteration 10: with minibatch training loss = 10.4 and accuracy of 0.68
Epoch 4, Overall loss = 8.31 and accuracy of 0.531
Epoch 5, Overall loss = 6.81 and accuracy of 0.704
Epoch 6, Overall loss = 6.68 and accuracy of 0.815
Iteration 20: with minibatch training loss = 0 and accuracy of 1
Epoch 7, Overall loss = 5.8 and accuracy of 0.778
Epoch 8, Overall loss = 5.47 and accuracy of 0.864
Epoch 9, Overall loss = 4.76 and accuracy of 0.901
Epoch 10, Overall loss = 3.88 and accuracy of 0.864
Iteration 30: with minibatch training loss = 1.54 and accuracy of 0.93
Epoch 11, Overall loss = 3.37 and accuracy of 0.914
Epoch 12, Overall loss = 3.28 and accuracy of 0.901
Epoch 13, Overall loss = 2.56 and accuracy of 0.926
Iteration 40: with minibatch training loss = 3.35 

(1.0761806460810297, 0.98765432098765427)

In [49]:
print('Training')
run_model(sess,y_out,mean_loss,x_train,y_train,1,40)
print('Validation')
run_model(sess,y_out,mean_loss,x_val,y_val,1,10)
print('Test')
run_model(sess,y_out,mean_loss,x_test,y_test,1,10)

Training
Epoch 1, Overall loss = 1 and accuracy of 0.988
Validation
Epoch 1, Overall loss = 3.68 and accuracy of 0.5
Test
Epoch 1, Overall loss = 13.7 and accuracy of 0.6


(13.742609024047852, 0.59999999999999998)